# Dask Flood Mapper

In [ ]:
from dask_flood_mapper import flood
from bokeh.models import FixedTicker
import hvplot.xarray

In [ ]:
import dask
dask.config.set(temporary_directory='/tmp')
from dask.distributed import Client, wait
client = Client(processes=False, threads_per_worker=2,
                n_workers=1, memory_limit='20GB')

In [ ]:
time_range = "2022-10-11/2022-10-25"
bbox = [12.3, 54.3, 13.1, 54.6]
fd = flood.decision(bbox=bbox, datetime=time_range).compute()
fd

In [ ]:
import os
import pystac_client
from odc import stac as odc_stac
from rasterio.enums import Resampling
import numpy as np

crs = "EPSG:4326"  # Coordinate Reference System - World Geodetic System 1984 (WGS84) in this case
res = 0.00018  # 20 meter in degree
os.environ["AWS_NO_SIGN_REQUEST"] = "YES"
wcover_catalog = pystac_client.Client.open("https://services.terrascope.be/stac/")
search = wcover_catalog.search(
    collections="urn:eop:VITO:ESA_WorldCover_10m_2021_AWS_V2", bbox=bbox
)

items_wcover = search.item_collection()
wcover_dc = (
    odc_stac.load(
        items_wcover,
        crs=4326,
        resolution=res,
        bbox=bbox,
    )
    .squeeze("time")
    .drop_vars("time")
    .rename_dims({"longitude": "x", "latitude": "y"})
    .rename_vars({"ESA_WORLDCOVER_10M_MAP": "wcover", "longitude": "x", "latitude": "y"})
    .transpose("y", "x")
)
wcover = wcover_dc.rio.reproject_match(fd, resampling=Resampling.bilinear, nodata=-999).wcover

In [ ]:
wcover = wcover.astype(int)

In [ ]:
fd = fd.where(wcover != 80)
fd

In [ ]:
colorbar_opts = {
    "major_label_overrides": {
        0: "non-flood",
        1: "flood",
    },
    "ticker": FixedTicker(ticks=[0, 1]),
}
fd.hvplot.image(
    x="x", y="y", rasterize=True, geo=True, cmap=["rgba(0, 0, 1, 0.1)", "darkred"]
).opts(frame_height=400, colorbar_opts={**colorbar_opts})

In [ ]:
client.close()